In [ ]:
!pip install timm
!python -c "from timm import list_models; print(list_models(pretrained=True)[:5])"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm
from PIL import Image
from torch import nn
from torchvision.transforms import ToTensor,Compose, Resize, Normalize
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
import os
from PIL import Image
import timm

In [ ]:
# File paths
file_path_normal = '/content/drive/MyDrive/Projetos/PoleProjet/data/images/normal/'
file_path_abnormal = '/content/drive/MyDrive/Projetos/PoleProjet/data/images/abnormal_augmented/'
# filepath = "/content/drive/MyDrive/Projects/data/images/abnormal"

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        # Define default transformations: Resize, ToTensor, and Normalize
        self.transform = transform if transform else Compose([
            Resize((256, 256)),  # Resize images to a fixed size
            ToTensor(),  # Convert images to tensor
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')  # Convert image to RGB
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

def load_images(file_path, label):
    images_paths = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith(('png', 'jpg', 'jpeg'))]  # Ensure it processes image files only
    labels = [label] * len(images_paths)
    return images_paths, labels

In [ ]:
# Load and label images
normal_images, normal_labels = load_images(file_path_normal, 0)  # 0 for normal
abnormal_images, abnormal_labels = load_images(file_path_abnormal, 1)  # 1 for abnormal

# Combine datasets
images = normal_images + abnormal_images
labels = normal_labels + abnormal_labels
#
# Split into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Create custom datasets
train_dataset = ImageDataset(train_images, train_labels)
test_dataset = ImageDataset(test_images, test_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
img_test = file_path_normal + 'normal_3747.png'

# Open the image using Pillow
image = Image.open(img_test)  # Replace "example.jpg" with your image file path

# Convert the image to a NumPy array
image_array = np.array(image)

# Now, you can work with the image as a NumPy array
print(image_array.shape)  # Print the shape of the array

In [ ]:
# Define the number of classes for binary classification
NUM_FINETUNE_CLASSES = 2  # True or False

# Create the model
model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=NUM_FINETUNE_CLASSES)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

### Train CNN

In [ ]:
# Assuming `train_labels` is a list or numpy array of your training labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
# print(class_weights)
# Modify the loss function to include the class weights

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

# Number of epochs for training
num_epochs = 50  # You can adjust this

model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    running_loss = 0.0

    # Initialize tqdm progress bar
    loop = tqdm(train_loader, total=len(train_loader), leave=False)

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Update the progress bar description
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss/len(train_loader))

    loop.close()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

print('Finished Training')

### Load and Test Model

In [ ]:
# Function to get predictions and true labels
def get_all_preds_labels(model, data_loader):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

# Get predictions and true labels
predictions, true_labels = get_all_preds_labels(model, test_loader)

# Calculate the confusion matrix
conf_mat = confusion_matrix(true_labels, predictions)

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', ax=ax, square=True)
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
ax.set_title('Confusion Matrix')
plt.show()


In [ ]:
a = []
for file in os.listdir(file_path_abnormal):
  a.append(file)
len(a)*0.2

In [ ]:
t_1_p_0 = sorted([test_dataset.file_paths[i].split('/')[-1] for i in range(len(predictions)) if (predictions[i]==0 and true_labels[i]==1)])
t_1_p_0

### Model Size

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def model_size(model):
    # Get the model's parameters
    params = list(model.parameters())

    # Calculate the size of the parameters in bytes
    total_size = sum(p.numel() * p.element_size() for p in params)

    # Convert bytes to megabytes (MB)
    total_size_mb = total_size / (1024 ** 2)

    return total_size_mb

In [ ]:
num_params = count_parameters(model)
print(f"Number of parameters in the model: {num_params}")

model_size_mb = model_size(model)
print(f"Model size: {model_size_mb:.2f} MB")